# 제 7회 기출 문제

## 작업형 제 1 유형

### ✏️ 결측치가 있는 행을 제거한 후 학생이 가장 많이 수강한 과목을 찾고, 해당 과목 점수를 표준화(스탠더드 스케일)한 다음 가장 큰 표준화된 값을 구하시오.(반올림하여 소수 셋째 자리까지 계산)

In [1]:
import pandas as pd
df = pd.read_csv('./data/student_assessment.csv')
print(df.head())

   id_assessment  student_id  study_period_days  score
0            103           0                 22   87.0
1             93           1                 70   92.0
2             15           2                 22    2.0
3            107           3                 22   51.0
4             72           4                 22   73.0


In [2]:
# 1. 결측치 제거
## dropna()를 활용해 결측치가 있는 모든 행 제거
## 결측치 제거 전후의 데이터프레임 크기를 출력해 제거된 행의 수 확인
print(df.shape)
df = df.dropna()
print(df.shape)

# 2. 가장 많이 수강한 과목 필터링
## value_counts() 사용해 각 과목별 수강한 학생의 수를 계산하고, idxmax()로 가장 많이 수강한 과목 ID 찾기
print(df['id_assessment'].value_counts())
id = df['id_assessment'].value_counts().idxmax()
print(id)
cond = df['id_assessment'] == id
df = df[cond]

# 3. 과목 점수 스탠더드 스케일
## 사이킷런의 StandardScaler 사용해 과목 점수를 스탠더드 스케일로 변환
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['score'] = scaler.fit_transform(df[['score']])
## 주의 : StandardScaler가 입력 데이터를 2차원 배열 형태로 처리해야 하기 때문에 대괄호 2개 사용
## df['score'] : Series 객체를 반환 (1차원 데이터)
## df[['score']] : DataFrame 객체를 반환 (2차원 데이터(하나의 열만 포함))

# 4. 가장 큰 값
## max() 사용해 가장 큰 값을 찾고, 이를 round()로 소수점 셋째 자리까지 반올림해 출력
print(round(df['score'].max(), 3))

(2565, 4)
(2544, 4)
id_assessment
12     33
47     32
8      31
113    29
39     28
       ..
77     11
18     10
43     10
83      9
71      9
Name: count, Length: 133, dtype: int64
12
2.183


---

#### 💡 idxmax() vs max()
- idxmax()

    - 목적: 최대값을 가지는 인덱스를 반환
    
    - 적용 대상: Series 또는 DataFrame
    
    - 결과: 최대값이 위치한 인덱스(또는 행/열 이름)

-  max()

    - 목적: 최대값을 반환

    - 적용 대상: Series 또는 DataFrame

    - 결과: Series에서는 최대값 자체, DataFrame에서는 축(axis)에 따라 최대값의 집합

<br>

|기능|idxmax()|max()|
|:-:|:-:|:-:|
|반환값|최대값의 인덱스 또는 행/열 이름|최대값 자체|
|주요 목적|최대값이 어디에 위치하는지 찾을 때|최대값의 크기를 확인할 때|
|적용 가능 대상|Series와 DataFrame|Series와 DataFrame|

---

### ✏️ DE1 ~ DE77 컬럼 중 주가지수의 종가 'close'와 가장 상관 관계가 높은 변수를 찾아 해당 변수의 평균값을 구하시오. (반올림하여 소수 넷째 자리까지 계산)

In [3]:
import pandas as pd
df = pd.read_csv('./data/stock_market.csv')
print(df.head())

        DE1       DE2       DE3       DE4       DE5       DE6       DE7  \
0  1.764052  0.400157  0.978738  2.240893  1.867558 -0.977278  0.950088   
1 -0.578850 -0.311553  0.056165 -1.165150  0.900826  0.465662 -1.536244   
2 -0.098453 -0.663478  1.126636 -1.079932 -1.147469 -0.437820 -0.498032   
3 -0.493320 -0.542861  0.416050 -1.156182  0.781198  1.494485 -2.069985   
4  0.693773 -0.159573 -0.133702  1.077744 -1.126826 -0.730678 -0.384880   

        DE8       DE9      DE10  ...      DE71      DE72      DE73      DE74  \
0 -0.151357 -0.103219  0.410599  ...  0.729091  0.128983  1.139401 -1.234826   
1  1.488252  1.895889  1.178780  ...  1.117016 -1.315907 -0.461585 -0.068242   
2  1.929532  0.949421  0.087551  ... -1.540797  0.063262  0.156507  0.232181   
3  0.426259  0.676908 -0.637437  ...  1.658131 -0.118164 -0.680178  0.666383   
4  0.094352 -0.042171 -0.286887  ... -0.206904  0.880179 -1.698106  0.387280   

       DE75      DE76      DE77        open       close    volume  


In [4]:
# 1. close 와의 상관 관계(절대값)
## corr()는 데이터프레임의 모든 숫자형 변수 간의 상관 계수 계산
## 상관 계수는 -1 부터 1까지의 값을 가지며, -1에 가까울수록 강한 음의 상관관계, 1에 가까울수록 강한 양의 상관관계
## 0은 상관관계가 없음을 나타냄
## df.corr()['close'] 를 통해 'close' 컬럼과 다른 모든 컬럼 간의 상관 계수 계산 후 abs() 사용해 절대값 구하기
## 절대값 찾는 이유 : 상관 관계의 방향성(음/양)에 관계없이 크기만 고려하기 위함
df_corr = df.corr()['close'].abs()
print(df_corr)

# 2. 상관 관계가 높은 변수명
## loc['DE1':'DE77'] 사용해 DE1 부터 DE77 까지의 변수들 중에서 'close' 컬럼과의 상관 계수 계산
## 계산된 상관 계수들 중 가장 큰 값을 idxmax() 활용해 컬럼의 이름(변수명) 구하기
col = df_corr.loc['DE1':'DE77'].idxmax()
print(col)

# 3. 2번에서 구한 변수명의 평균값
## 2번에서 구한 변수명의 평균값 구한 후 결과값을 반올림하여 소수 넷째 자리까지 출력
print(round(df[col].mean(), 4))

DE1       0.034633
DE2       0.052457
DE3       0.021811
DE4       0.025823
DE5       0.001282
            ...   
DE76      0.012707
DE77      0.020981
open      0.996873
close     1.000000
volume    0.022469
Name: close, Length: 80, dtype: float64
DE14
-0.0004


### ✏️ IQR 을 이용해 이산화탄소(CO2) 이상치 수를 찾으시오

- Q1(제1사분위수) : 하위 25%값

- Q3(제3사분위수) : 상위 25%값

- IQR(사분위수 범위) : IQR = Q3 - Q1

- lower(하한선) : Q1 - 1.5 * IQR, 이 값보다 작은 데이터는 이상치로 간주

- upper(상한선) : Q3 + 1.5 * IQR, 이 값보다 큰 데이터는 이상치로 간주

In [5]:
import pandas as pd
df = pd.read_csv('./data/air_quality.csv')
print(df.head())

                  Date        CO         CO2         O2
0  2024-01-01 00:00:00  0.549671  383.037632  21.248286
1  2024-01-01 01:00:00  0.486174  392.362513  21.183324
2  2024-01-01 02:00:00  0.564769  385.065473  19.963480
3  2024-01-01 03:00:00  0.652303  402.760451  21.479584
4  2024-01-01 04:00:00  0.476585  429.929463  19.409917


In [6]:
# 1. IQR 계산
Q1 = df['CO2'].quantile(0.25)
Q3 = df['CO2'].quantile(.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)

# 2. 상한 및 하한 게산
upper = Q3 + 1.5 * IQR
lower = Q1 - 1.5 * IQR

# 3. 이상치 식별
cond1 = df['CO2'] > upper
cond2 = df['CO2'] < lower
이상치 = df[cond1 | cond2]
print(이상치)

# 4. 이상치 수
## len() 사용해 식별된 이상치 개수 구하기
print(len(이상치))

383.8502072254157 418.7784333384595 34.92822611304376
                     Date        CO         CO2         O2
88    2024-01-04 16:00:00  0.447024  603.471090  20.712496
155   2024-01-07 11:00:00  0.428565  588.211313  22.013521
185   2024-01-08 17:00:00  0.571400  472.852730  21.667225
196   2024-01-09 04:00:00  0.411614  607.631613  21.072704
228   2024-01-10 12:00:00  0.428470  595.429242  20.319365
...                   ...       ...         ...        ...
9908  2025-02-16 20:00:00  0.606182  595.889840  21.572550
9938  2025-02-18 02:00:00  0.373704  627.451289  20.840325
9953  2025-02-18 17:00:00  0.403922  639.521653  19.060725
9962  2025-02-19 02:00:00  0.568850  328.099193  21.601589
9966  2025-02-19 06:00:00  0.451337  646.129551  20.328131

[304 rows x 4 columns]
304
